<a href="https://colab.research.google.com/github/Ashwin1234/ML_OCR_project1/blob/main/HandWritten_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random 
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.utils import np_utils
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from google.colab.patches import cv2_imshow

In [73]:
LB=LabelBinarizer()

In [39]:
!unzip Images_1.zip -d images_1

Archive:  Images_1.zip
   creating: images_1/images/
  inflating: images_1/images/TRAIN_00001.jpg  
  inflating: images_1/images/TRAIN_00002.jpg  
  inflating: images_1/images/TRAIN_00003.jpg  
  inflating: images_1/images/TRAIN_00004.jpg  
  inflating: images_1/images/TRAIN_00005.jpg  
  inflating: images_1/images/TRAIN_00006.jpg  
  inflating: images_1/images/TRAIN_00007.jpg  
  inflating: images_1/images/TRAIN_00008.jpg  
  inflating: images_1/images/TRAIN_00009.jpg  
  inflating: images_1/images/TRAIN_00010.jpg  
  inflating: images_1/images/TRAIN_00011.jpg  
  inflating: images_1/images/TRAIN_00012.jpg  
  inflating: images_1/images/TRAIN_00013.jpg  
  inflating: images_1/images/TRAIN_00014.jpg  
  inflating: images_1/images/TRAIN_00015.jpg  
  inflating: images_1/images/TRAIN_00016.jpg  
  inflating: images_1/images/TRAIN_00017.jpg  
  inflating: images_1/images/TRAIN_00018.jpg  
  inflating: images_1/images/TRAIN_00019.jpg  
  inflating: images_1/images/TRAIN_00020.jpg  
  infla

In [65]:
!unzip test_images.zip -d test_images

Archive:  test_images.zip
   creating: test_images/test_images/
  inflating: test_images/test_images/TEST_0001.jpg  
  inflating: test_images/test_images/TEST_0002.jpg  
  inflating: test_images/test_images/TEST_0003.jpg  
  inflating: test_images/test_images/TEST_0004.jpg  
  inflating: test_images/test_images/TEST_0005.jpg  
  inflating: test_images/test_images/TEST_0006.jpg  
  inflating: test_images/test_images/TEST_0007.jpg  
  inflating: test_images/test_images/TEST_0008.jpg  
  inflating: test_images/test_images/TEST_0009.jpg  
  inflating: test_images/test_images/TEST_0010.jpg  
  inflating: test_images/test_images/TEST_0011.jpg  
  inflating: test_images/test_images/TEST_0012.jpg  
  inflating: test_images/test_images/TEST_0013.jpg  
  inflating: test_images/test_images/TEST_0014.jpg  
  inflating: test_images/test_images/TEST_0015.jpg  
  inflating: test_images/test_images/TEST_0016.jpg  
  inflating: test_images/test_images/TEST_0017.jpg  
  inflating: test_images/test_image

In [76]:
!unzip data.zip -d data


Archive:  data.zip
   creating: data/data/
   creating: data/data/images/
  inflating: data/data/images/TRAIN_00001.jpg  
  inflating: data/data/images/TRAIN_00002.jpg  
  inflating: data/data/images/TRAIN_00003.jpg  
  inflating: data/data/images/TRAIN_00004.jpg  
  inflating: data/data/images/TRAIN_00005.jpg  
  inflating: data/data/images/TRAIN_00006.jpg  
  inflating: data/data/images/TRAIN_00007.jpg  
  inflating: data/data/images/TRAIN_00008.jpg  
  inflating: data/data/images/TRAIN_00009.jpg  
  inflating: data/data/images/TRAIN_00010.jpg  
  inflating: data/data/images/TRAIN_00011.jpg  
  inflating: data/data/images/TRAIN_00012.jpg  
  inflating: data/data/images/TRAIN_00013.jpg  
  inflating: data/data/images/TRAIN_00014.jpg  
  inflating: data/data/images/TRAIN_00015.jpg  
  inflating: data/data/images/TRAIN_00016.jpg  
  inflating: data/data/images/TRAIN_00017.jpg  
  inflating: data/data/images/TRAIN_00018.jpg  
  inflating: data/data/images/TRAIN_00019.jpg  
  inflating: d

In [133]:
dir = '/content/data/data/images'
img_size=32
train_images=[]
for j in os.listdir(dir):
  img = cv2.imread(os.path.join(dir,j),0)
  img = cv2.resize(img,(img_size,img_size))
  train_images.append(img)

In [134]:
len(train_image)

1000

In [135]:
dir = '/content/data'
df=pd.read_excel(os.path.join(dir,'names.xlsx'),names=['images','labels'])
df.head()

,images,labels
0,TRAIN_00002.jpg,SIMON
1,TRAIN_00003.jpg,BENES
2,TRAIN_00004.jpg,LA LOVE
3,TRAIN_00005.jpg,DAPHNE
4,TRAIN_00006.jpg,LUCIE


In [136]:
train_labels=df['labels']

In [137]:
train_X = []
train_Y = []
for image in train_image:
  train_X.append(image)
for label in train_labels:
  train_Y.append(label)

In [138]:
LB = LabelBinarizer()
train_Y = LB.fit_transform(train_Y)


In [139]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [152]:
def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray ,127,255,cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        roi = gray[y:y + h, x:x + w]
        cv2_imshow(roi)

        thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (32, 32), interpolation = cv2.INTER_CUBIC)
        thresh = thresh.astype("float32") / 255.0
        thresh = np.expand_dims(thresh, axis=-1)
        thresh = thresh.reshape(1,32,32,1)
        ypred = thresh
      
        ypred = LB.inverse_transform(ypred)
        [x] = ypred
        letters.append(x)
    return letters, image


In [153]:
dir = '/content/test_images/test_images'
test_images=[]
for j in os.listdir(dir):
  test_images.append(os.path.join(dir,j))

In [154]:
letters,image=get_letters(test_images[0])
#print(letters)